# DS-GA 1011 Fall 2018 Lab 5
# Intrinsic Evaluation of Word Vectors

## Part 0: Setup

Before starting, make sure you've downloaded the following: 
1. [GloVe vectors](https://nlp.stanford.edu/projects/glove/): We'll use the 6B, 50D version so download glove.6B.zip (822MB) from the website (or `wget http://nlp.stanford.edu/data/glove.6B.zip` )
2. [fastText vectors](https://fasttext.cc/docs/en/english-vectors.html): We'll use the 1M, 300D version (650MB) (`wget https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki-news-300d-1M.vec.zip`)



Now let's load a set of 50D word vectors from GloVe. `glove_home` below specifies the location of the unzipped file. `words_to_load` specifies how many word vectors we want to load. The words are saved in frequency order, so specifying 50,000 means that we only want to work with the 50,000 most frequent words from the source corpus. You can load up to 400,000 words.

In [1]:
import pprint
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
pp = pprint.PrettyPrinter(indent=4)
from operator import itemgetter

In [2]:
glove_home = './'
words_to_load = 50000

import numpy as np

with open(glove_home + 'glove.6B.50d.txt') as f:
    loaded_embeddings = np.zeros((words_to_load, 50))
    words = {}
    idx2words = {}
    ordered_words = []
    for i, line in enumerate(f):
        if i >= words_to_load: 
            break
        s = line.split()
        loaded_embeddings[i, :] = np.asarray(s[1:])
        words[s[0]] = i
        idx2words[i] = s[0]
        ordered_words.append(s[0])

Here's how to look up a word:

In [3]:
# loaded_embeddings: original embedding matrix, dim = (words_to_load, 50)
# words: a dictionary that maps word to its idx
# idx2words: a dictionary that maps idx to word
print(loaded_embeddings[words['potato']])

[-0.063054 -0.62636  -0.76417  -0.041484  0.56284   0.86432  -0.73734
 -0.70925  -0.073065 -0.74619  -0.34769   0.14402   1.4576    0.034688
  0.11224   0.13854   0.10484   0.60207   0.021777 -0.21802   0.087613
 -1.4234    1.0361    0.1509    0.13608  -0.2971   -0.90828   0.34182
  1.3367    0.16329   1.2374   -0.20113  -0.91532   1.4222   -0.1276
  0.69443  -1.1782    1.2072    1.0524   -0.11957  -0.1275    0.41798
 -0.9232   -0.1312    1.2696    1.2318    0.30061  -0.18854   0.15899
  0.0486  ]


## Part 1: Similarity Measure

Implement the function dot_similarity that returns the same similarity as the cosine_similarity in sklearn for the same inputs.

In [10]:
def sklearn_cosine_similarity(vec_one, vec_two):
    """
    Function that calculates the cosine similarity between two words
    """
    return float(cosine_similarity(np.array([vec_one,vec_two]))[0,1])


def handcraft_cosine_similarity(vec_one, vec_two):
    """
    Function that calculates the cosine similarity between two words
    """
    #TODO: fill in your code
    return 1.0

# Your handcraft_cosine_similarity should give (almost) same values as sklearn_cosine_similarity
print(handcraft_cosine_similarity(loaded_embeddings[words["good"]], loaded_embeddings[words["bad"]]))
print(sklearn_cosine_similarity(loaded_embeddings[words["good"]], loaded_embeddings[words["bad"]]))

print(handcraft_cosine_similarity(loaded_embeddings[words["good"]], loaded_embeddings[words["well"]]))
print(sklearn_cosine_similarity(loaded_embeddings[words["good"]], loaded_embeddings[words["well"]]))

print(handcraft_cosine_similarity(loaded_embeddings[words["good"]], loaded_embeddings[words["fish"]]))
print(sklearn_cosine_similarity(loaded_embeddings[words["good"]], loaded_embeddings[words["fish"]]))


1.0
0.7964893661716317
1.0
0.8510908646017762
1.0
0.4897107000383957


## Part 2: The Semantic Orientation Method

The __semantic orientation__ method of [Turney and Littman 2003](http://doi.acm.org/10.1145/944012.944013) is a method for automatically scoring words along some single semantic dimension like sentiment. It works from a pair of small seed sets of words that represent two opposing points on that dimension.

*Some code in this section was adapted from Stanford CS 224U*

In [5]:
def determine_coefficient(candidate_word, loaded_embeddings):
    # Here's a sample pair of seed sets:
    seed_pos = ['table', 'chair', 'lamp', 'curtain', 'desk']
    seed_neg = ['fish', 'bird', 'dog', 'cat', 'cow']
    
    # Let's look up the embeddings for these words.
    seed_pos_indices = [words[seed] for seed in seed_pos]
    seed_neg_indices = [words[seed] for seed in seed_neg]
    seed_pos_mat = loaded_embeddings[seed_pos_indices]
    seed_neg_mat = loaded_embeddings[seed_neg_indices]

    # Scoring words along the axis
    candidate = loaded_embeddings[words[candidate_word]]
    pos_sim = np.sum([cosine_similarity(np.array([candidate,reference]))[0,1] for reference in seed_pos_mat])
    neg_sim = np.sum([cosine_similarity(np.array([candidate,reference]))[0,1] for reference in seed_neg_mat])
    return pos_sim - neg_sim

In [23]:
determine_coefficient('abhorrent', loaded_embeddings)

-1.2916448442212511

And sort our vocabulary by its score along the axis. For now, we're only scoring frequent words, since this process can be slow.

In [33]:
scored_words = [(word, determine_coefficient(word, loaded_embeddings)) for word in ordered_words[1:10000]]
sorted_words = sorted(scored_words, key=itemgetter(1), reverse=True)

In [34]:
pp.pprint(sorted_words[:10])
pp.pprint(sorted_words[-10:])

[   ('panels', 2.0888931338753922),
    ('desk', 2.031519353296948),
    ('chairs', 1.9969309470439887),
    ('chair', 1.9807613618158875),
    ('slobodan', 1.9798640819000619),
    ('ceiling', 1.9240190533444927),
    ('doors', 1.9204824593800405),
    ('rotating', 1.8855537456237359),
    ('belgrade', 1.8764716706543156),
    ('columns', 1.8563476943420172)]
[   ('cow', -2.9569226842503795),
    ('breeding', -2.9796908223187222),
    ('breed', -2.985939176876923),
    ('bird', -3.0657022888194758),
    ('cats', -3.1399804835837362),
    ('cattle', -3.1442049121562579),
    ('whale', -3.1587269448292599),
    ('shark', -3.2280199929166615),
    ('sheep', -3.2573173088766909),
    ('pigs', -3.3829592988198574)]


Spend a few minutes exploring possible seed sets for other semantic dimensions. What works? What doesn't? Why?

## Part 3: Word Analogies


The word analogy task consists of questions like, “a is to b as c is to ?” As mentioned in the GloVe paper, the answer to this problem is the word that gives the max cosine similarity for equation emb(b) − emb(a) + emb(c).

In [35]:
def find_nearest_word(input_vec, k=5):
    """
    Function that returns the top k words whose embedding has the smallest cosine distance to the input_vec
    @param input_vec: embedding for a single word
    @param k: top k neighbours to return
    """
    #TODO: fill in your code
    return None


def word_analogy(word_a, word_b, word_c, k=5):
    """
    Function that solves problem word_a to word_b = word_c to ?
    @param word_a, word_b, word_c: string
    @param k: top k candidates to return
    """
    #TODO: fill in your code
    return None


# embedding algebra
print(find_nearest_word(loaded_embeddings[words["student"]] - loaded_embeddings[words["study"]], k=2))
print(find_nearest_word(loaded_embeddings[words["working-class"]] + loaded_embeddings[words["money"]], k=2))
print(find_nearest_word(loaded_embeddings[words["drunk"]] - loaded_embeddings[words["alcohol"]], k=5))


# Analogy
print(word_analogy("china", "chinese", "america"))
print(word_analogy("china", "beijing", "america"))
print(word_analogy("king", "male", "queen"))
print(word_analogy("athens", "greece", "berlin"))
print(word_analogy("dark", "darker", "soft"))

[('assaulting', 0.6012762284317517), ('bartender', 0.5673272156148943)]
[('wealthy', 0.78508652798614), ('poor', 0.7807281621143065)]
[('exclaimed', 0.5803865159891104), ('dejected', 0.580236670612162), ('heartbroken', 0.5751155810308969), ('grimly', 0.5561391408895854), ('bewildered', 0.5340116744760797)]
['american', 'young', 'many', 'popular']
['d.c.', 'york', 'angeles', 'hollywood']
['female', 'adult', 'girls', 'women']
['germany', 'german', 'europe']
['softer', 'texture', 'lean', 'thicker']


## Part 5: Fast Text

Now let's try loading Fast text vectors and analyse them in a similar way

In [7]:
ft_home = './'
words_to_load = 50000

import numpy as np

with open(ft_home + 'wiki-news-300d-1M.vec') as f:
    loaded_embeddings_ft = np.zeros((words_to_load, 300))
    words_ft = {}
    idx2words_ft = {}
    ordered_words_ft = []
    for i, line in enumerate(f):
        if i >= words_to_load: 
            break
        s = line.split()
        loaded_embeddings_ft[i, :] = np.asarray(s[1:])
        words_ft[s[0]] = i
        idx2words_ft[i] = s[0]
        ordered_words_ft.append(s[0])

Verifying the cosine similarity between fT vectors 

In [ ]:
print(sklearn_cosine_similarity(loaded_embeddings_ft[words["good"]], loaded_embeddings[words_ft["bad"]]))
print(sklearn_cosine_similarity(loaded_embeddings_ft[words["good"]], loaded_embeddings[words_ft["well"]]))
print(sklearn_cosine_similarity(loaded_embeddings_ft[words["good"]], loaded_embeddings[words_ft["fish"]]))

Analysing the semantic orientation

In [ ]:
scored_words = [(word, determine_coefficient(word, loaded_embeddings_ft)) for word in ordered_words[1:10000]]
sorted_words = sorted(scored_words, key=itemgetter(1), reverse=True)
pp.pprint(sorted_words[:10])
pp.pprint(sorted_words[-10:])

In [ ]:
print(find_nearest_word(loaded_embeddings_ft[words["student"]] - loaded_embeddings_ft[words["study"]], k=2))
print(find_nearest_word(loaded_embeddings_ft[words["working-class"]] + loaded_embeddings_ft[words["money"]], k=2))
print(find_nearest_word(loaded_embeddings_ft[words["drunk"]] - loaded_embeddings_ft[words["alcohol"]], k=5))


# Analogy
print(word_analogy("china", "chinese", "america"))
print(word_analogy("china", "beijing", "america"))
print(word_analogy("king", "male", "queen"))
print(word_analogy("athens", "greece", "berlin"))
print(word_analogy("dark", "darker", "soft"))

# Part 6: Visualize word vectors (HW)

In [ ]:
### TODO: TSNE plots

# More questions to think about:
- Can we analyse and quantify the difference in Glove and fastText vectors?
- If we only care about the nearest neighbour in a fixed set, will the neighbour with smallest L2 distance be the same neighbour that gives the max cosine similarity?
- Will we lose any information about embeddings if we normalize the embedding vectors? Why?
- Is cosine distance (1/cosine similarity) a valid distance metrics? Why?